In [0]:
from delta.tables import *

source_table='incremental_load.orders_data_schema.orders_stage'
target_table='incremental_load.orders_data_schema.orders_load'

In [0]:
stage_df=spark.read.table(source_table)
print('read')
stage_df.show()

In [0]:
if not spark.catalog.tableExists(target_table):
    stage_df.write.format('delta').saveAsTable(target_table)
else:
    target_delta=DeltaTable.forName(spark,target_table)

    join_cond='target.tracking_num==stage.tracking_num'

    target_delta.alias('target').merge(stage_df.alias('stage'),join_cond)\
        .whenMatchedDelete()\
        .execute()

    stage_df.write.format('delta').mode('append').saveAsTable(target_table)
    print('written')
